In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

class PyTorchGPTLab:
    def __init__(self):
        self.tokenizer = None
        self.model = None
        self.is_loaded = False
        
        # --- UI Components ---
        self.header = widgets.HTML("<h2>🔥 PyTorch & GPT-2: Training Loss Lab</h2>")
        
        self.input_text = widgets.Text(
            value='Fine-tuning GPT models',
            description='Input Text:',
            placeholder='Type sentence here...',
            layout=widgets.Layout(width='400px')
        )
        
        self.btn_run = widgets.Button(
            description="Run Forward Pass",
            button_style='danger', # Red/Orange for PyTorch theme
            icon='play'
        )
        
        self.btn_load = widgets.Button(
            description="Initialize Model",
            button_style='primary',
            icon='download'
        )
        
        self.output_area = widgets.Output()
        
        # --- Event Listeners ---
        self.btn_run.on_click(self.run_forward_pass)
        self.btn_load.on_click(self.load_model)

    def load_model(self, b):
        """Loads the GPT-2 model and tokenizer."""
        with self.output_area:
            clear_output()
            print("⏳ Initializing PyTorch GPT-2 Model... (This may take a moment)")
            
            try:
                self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
                self.model = GPT2LMHeadModel.from_pretrained("gpt2")
                # Important: Set to train mode for dropout/grad tracking (though we use no_grad for demo)
                self.model.train() 
                
                self.is_loaded = True
                print("✅ Model Loaded Successfully!")
                print("Ready to run forward passes.")
                
                # Hide load button after loading
                self.btn_load.layout.display = 'none'
                
            except Exception as e:
                print(f"❌ Error loading model: {e}")

    def run_forward_pass(self, b):
        """Executes the specific requirement logic."""
        if not self.is_loaded:
            with self.output_area:
                print("⚠️ Please initialize the model first.")
            return

        text = self.input_text.value
        if not text:
            return

        with self.output_area:
            clear_output()
            print(f"📝 Input Text: \"{text}\"")
            print("-" * 50)
            
            try:
                # --- REQUIREMENT LOGIC START ---
                
                # 1. Tokenize
                # Convert text strings into tensor integers
                input_ids = self.tokenizer(text, return_tensors="pt").input_ids
                
                print(f"🔹 Tensor Shape: {tuple(input_ids.shape)}")
                print(f"🔹 Input IDs: {input_ids.tolist()[0]}")
                
                # 2. Forward Pass
                # We pass 'labels' to trigger loss calculation
                # In standard causal training, labels = input_ids (predict next token)
                outputs = self.model(input_ids=input_ids, labels=input_ids)
                
                # 3. Extract Loss
                loss = outputs.loss
                loss_value = loss.item()
                
                # --- REQUIREMENT LOGIC END ---
                
                print("-" * 50)
                print(f"📉 Training Loss: {loss_value:.4f}")
                print("-" * 50)
                
                if loss_value < 3.0:
                    print("✅ Low Loss: The model finds this sentence predictable.")
                else:
                    print("⚠️ High Loss: The model finds this sentence surprising.")
                    
            except Exception as e:
                print(f"❌ Computation Error: {e}")

    def render(self):
        """Displays the UI."""
        display(widgets.VBox([
            self.header,
            self.btn_load,
            widgets.HBox([self.input_text, self.btn_run]),
            self.output_area
        ]))

# --- Run the Lab ---
if __name__ == "__main__":
    app = PyTorchGPTLab()
    app.render()

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\users\user\AppData\Roaming\Python\Python312\site-packages\torch\lib\c10.dll" or one of its dependencies.